In [1]:
import pandas as pd
import altair as alt

In [2]:
alt.data_transformers.disable_max_rows() # Disable 5_000 rows limit
data = pd.read_csv("../leaderboard_code4rena.csv") # Set path accordingly

In [3]:
data

,period,handle,is_team,prize_money,total_reports,high_all,high_solo,med_all,med_solo,gas_all
0,2022,WatchPug,True,365004.16,409,68,24,119,44,154
1,2022,cmichel,False,235136.67,135,30,5,60,19,3
2,2022,hyh,False,176095.70,200,23,1,66,19,20
3,2022,IllIllI,False,167108.96,174,17,2,69,14,47
4,2022,leastwood,False,163097.64,109,37,9,53,25,1
...,...,...,...,...,...,...,...,...,...,...
1284,ALL TIME,solgryn,False,5.77,4,0,0,0,0,4
1285,ALL TIME,aez121,False,0.10,1,0,0,1,0,0
1286,ALL TIME,ad3sh_,False,0.00,2,0,0,0,0,0
1287,ALL TIME,Kumpirmafyas,False,0.00,2,0,0,0,0,0


In [23]:
plt_data = data.copy()

### Period dropdown
Filter from which period the data should be taken. These are the same options as the [leaderboard](https://code4rena.com/leaderboard/) entries from the Code4rena website.

In [24]:
period_dropdown = alt.binding_select(options=list(plt_data["period"].unique()), name='Select a period:')
period_selector = alt.selection_single(
    fields=['period'], 
    bind=period_dropdown, 
    name="period_selector", 
    init={'period':'ALL TIME'}
)

In [25]:
is_team_legend=['Solo', 'Team']
is_team_colors = ['#2c385c', '#ffb412']
plt_data["is_team"] = plt_data["is_team"].map(lambda x: is_team_legend[1 if x else 0])

for col in plt_data.columns[3:]:
    plt_data['sum_' + col] = plt_data.groupby(['period', 'is_team'])[col].transform('sum')

In [61]:
total_prize_base = alt.Chart(plt_data).transform_joinaggregate(
    total_prize_money='sum(prize_money)'
).transform_calculate(
    prize_share='datum.sum_prize_money / datum.total_prize_money'
).encode(
    theta=alt.Theta("sum(prize_money):Q", stack=True),
    color=alt.Color(
        "is_team:N",
        scale=alt.Scale(range=is_team_colors),
        legend=alt.Legend(
            title='Wardens',
            orient='top-right',
            labelFontSize=14, 
            titleFontSize=14,
            symbolType='square',
        )
    ),
    tooltip=['is_team:N', alt.Tooltip('sum_prize_money:Q', format='$,'), alt.Tooltip('prize_share:Q', format='.2%')]
)

In [62]:
pie_radius = 200

In [63]:
total_prize_pie = total_prize_base.mark_arc(
    outerRadius=pie_radius
)

In [64]:
total_prize_labels = total_prize_base.mark_text(
    radius=pie_radius*1.15,
    size=pie_radius/10
).encode(
    text=alt.Text("prize_share:N", format='.2%')
)

In [65]:
(total_prize_pie + total_prize_labels).properties(
    title="Total prize money ($USD)",
    width=2.5*pie_radius,
    height=2.5*pie_radius
).add_selection(
    period_selector
).transform_filter(
    period_selector
)

alt.LayerChart(...)